# Requisições à API dos Circulares da USP

A API dos circulares da USP, disponível em https://uspdigital.usp.br/mobile/servicos/sptrans/posicoes, indica os veículos que estão trafegando no momento na universidade. Tais dados são úteis para o rastreamento dos ônibus e a coleta de dados geral a respeito dos tempos de viagem, por exemplo.

In [1]:
import requests
import json
import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.dates as md

In [2]:
class Bus:
    def __init__(self, bus_id, bus_line, last_register, position):
        self.bus_id = bus_id
        self.bus_line = bus_line
        self.last_register = last_register
        self.position = position

In [3]:
class State:
    def __init__(self, url):
        self.bus_list = []
        self.data_register = datetime.datetime.now()
        
        dict_api_states = self.request_api_state(url)
        self.create_vehicles(dict_api_states)
        
    def request_api_state(self, url):
        """ A função tem o objetivo de realizar uma requisição a API de endereço url, 
        retornando um dicionário com o estado atual dos ônibus na universidade """
        
        try:
            s_dict = requests.get(url).text 
            dict = json.loads(s_dict)
        except Exception as e:
            dict = []
            print(e)
        
        return dict
    
    def create_vehicles(self, dict):
        for bus_line in dict:
            for bus in bus_line["vs"]:
                bus_position = (bus["px"], bus["py"])
                obj_bus = Bus(bus["p"], bus_line["l"], bus["ta"], bus_position)
                
                self.bus_list.append(obj_bus)
    
    def get_vehicles(self):
        return self.bus_list
    
    def dump_list(self):
        i = 0
        str = '{"data_register": "' + self.data_register.strftime("%Y-%m-%dT%H:%M:%SZ") + '", '
        str += '"buses": ['
        for bus in self.bus_list:
            str += json.dumps(vars(bus))
            if i < len(self.bus_list) - 1:
                str += ','
            i += 1
            
        return str + ']}'

## Registro de dados de estados

Um estado da API, representado por uma lista de ônibus, é bastante informativo no quesito de dados necessários à análise de questões referentes aos veículos. O registro de estado pode, então, ser realizado em um arquivo JSON em um dado período de tempo.

In [4]:
def data_register(duration_time, x, url):
    """ Dado um tempo duration_time em segundos, registra os dados de estados em um arquivo JSON
    a cada x segundos """
    
    now = datetime.datetime.now()
    duration = datetime.timedelta(seconds=duration_time)
    endtime = now + duration
    
    while datetime.datetime.now() <= endtime:
        curr_state = State(url)
        with open('data/states.json', 'a') as f:
            f.write(curr_state.dump_list() + '\n')
        time.sleep(x)

## Leitura de dados escritos

A partir do registro de dados realizado pela função data_register, precisamos ler o histórico do arquivo para recuperar a lista de estados. Isso é feito pela função data_read, que, dado o nome do arquivo, retorna a lista que representa o conjunto de todos os estados registrados naquele arquivo.

In [5]:
def data_read(file):
    """ Retorna uma lista de estados dado um arquivo file que os contenha no formato definido na função
    data_register """
    
    states = []
    with open(file, 'r') as f:
        for line in f.readlines():
            state = json.loads(line)
            states.append(state)
    return states

# Análise de Frotas

Com os registros de vários estados ao longo de um dia, podemos verificar a quantidade de ônibus rodando ao longo de cada horário.

In [6]:
def fix_hour(date, offset):
    date_offset = datetime.timedelta(hours=offset)
    return date+date_offset

## Gerando os gráficos por linha

Considerando as linhas "8012", "8022" e "8032", temos a função que é capaz de calcular, dado o vetor dos estados da API, a quantidade de ônibus por tempo, dada uma linha fixa.

In [7]:
def amount_buses(states, line=None):
    """ Calcula a quantidade amount de ônibus por tempo dada a linha line """
    amount = []
    for state in states:
        list_buses = state["buses"]
        
        if line == "8012":
            list_buses = list(filter(lambda x: x["bus_line"] == 8012, list_buses))
        elif line == "8022":
            list_buses = list(filter(lambda x: x["bus_line"] == 8022, list_buses))
        elif line == "8032":
            list_buses = list(filter(lambda x: x["bus_line"] == 8032, list_buses))
                
        amount.append(len(list_buses))

    return amount

In [8]:
def times_register(states):
    """ Calcula os tempos de registro de cada estado """ 
    hours = []
    for state in states:
        register_time = datetime.datetime.strptime(state["data_register"], "%Y-%m-%dT%H:%M:%SZ")
        hours.append(register_time)
        
    for i in range(len(hours)):
        hours[i] = fix_hour(hours[i], -3)
        
    return hours

In [9]:
def plot_buses(states, category=None):   
    """ Plota os gráficos de dados dos ônibus de acordo com o conjunto de estados da API e a"
    categoria de gráfico correspondente """
    hours = times_register(states)
    amount_all = amount_buses(states)
    amount_c1 = amount_buses(states, "8012")
    amount_c2 = amount_buses(states, "8022")
    amount_c3 = amount_buses(states, "8032")
    
    fig, ax = plt.subplots(figsize=(12,8))
    
    if category == "8012":
        plt.plot(hours, amount_c1, color="red", label="Linha 8012")
        plt.yticks(range(0, max(amount_c1), 2))
    elif category == "8022":
        plt.plot(hours, amount_c2, color="green", label="Linha 8022")
        plt.yticks(range(0, max(amount_c2), 2))
    elif category == "8032":
        plt.plot(hours, amount_c3, color="blue", label="Linha 8032")
        plt.yticks(range(0, max(amount_c3), 2))
    else:
        plt.plot(hours, amount_c1, color="red", label="Linha 8012")
        plt.plot(hours, amount_c2, color="green", label="Linha 8022")
        plt.plot(hours, amount_c3, color="blue", label="Linha 8032")
        plt.plot(hours, amount_all, color="black", label="Todas as linhas")
        plt.yticks(range(0, max(amount_all), 2))
        plt.axhline(y=18, color="black", linestyle='--')
        
    ax.set_xlim(min(hours)-datetime.timedelta(hours=1),
                max(hours)+datetime.timedelta(hours=1))
    ax.legend(loc="upper right")

    ax.xaxis.set_major_locator(md.HourLocator(interval = 1))
    ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M:%S'))

    fig.autofmt_xdate()

    plt.show()